# Cohort Analysis with SQL

In [67]:
import pymysql
import pandas as pd

In [2]:
conn = pymysql.connect(host='localhost', user='root', password=str(input()), db='sales') 
curs = conn.cursor(pymysql.cursors.DictCursor)

In [71]:
# 데이터 결측값 제거, 중복값 제거, 음수, 0인 값 제거 (전처리)
sql_p_data = '''
      WITH p_data AS (
            SELECT *
            FROM online_sales
            WHERE CustomerID IS NOT NULL AND
                  Quantity > 0 AND
                  UnitPrice > 0
            GROUP BY 1,2,3,4,5,6,7,8
      )
''' 

# 고객 코호트 구성을 위한 첫구매일 확인
sql_first_purchase = '''
      first_purchase AS (
            SELECT CustomerID, MIN(DATE(InvoiceDate)) AS CohortDate
            FROM p_data
            GROUP BY CustomerID
      )
'''

# 데이터 조인
sql_cohort_group = '''
      cohort_merged AS (
            SELECT d.CustomerID, 
                  f.CohortDate, 
                  (YEAR(InvoiceDate)-YEAR(f.CohortDate))*12+(MONTH(InvoiceDate)-MONTH(f.CohortDate))+1 AS CohortIndex, 
                  DATE_FORMAT(f.CohortDate, '%Y-%m') AS CohortGroup
            FROM p_data d LEFT JOIN first_purchase f
            ON d.CustomerID = f.CustomerID
      )
'''

# 그룹화
sql_cohort_anaysis = '''
      cohort_grouped AS (
            SELECT CohortGroup, CohortIndex, COUNT(DISTINCT CustomerID) AS CustomerCount
            FROM cohort_merged
            GROUP BY 1,2
      )
'''

# Pivot
sql_pivot = '''
      cohort_pivot AS (
      SELECT CohortGroup,
             CASE WHEN CohortIndex = 1 THEN CustomerCount ELSE 0 END AS I,
             CASE WHEN CohortIndex = 2 THEN CustomerCount ELSE 0 END AS II,
             CASE WHEN CohortIndex = 3 THEN CustomerCount ELSE 0 END AS III,
             CASE WHEN CohortIndex = 4 THEN CustomerCount ELSE 0 END AS IV,
             CASE WHEN CohortIndex = 5 THEN CustomerCount ELSE 0 END AS V,
             CASE WHEN CohortIndex = 6 THEN CustomerCount ELSE 0 END AS VI,
             CASE WHEN CohortIndex = 7 THEN CustomerCount ELSE 0 END AS VII,
             CASE WHEN CohortIndex = 8 THEN CustomerCount ELSE 0 END AS VIII,
             CASE WHEN CohortIndex = 9 THEN CustomerCount ELSE 0 END AS IX,
             CASE WHEN CohortIndex = 10 THEN CustomerCount ELSE 0 END AS X,
             CASE WHEN CohortIndex = 11 THEN CustomerCount ELSE 0 END AS XV,
             CASE WHEN CohortIndex = 12 THEN CustomerCount ELSE 0 END AS XVI,
             CASE WHEN CohortIndex = 13 THEN CustomerCount ELSE 0 END AS XVII
      FROM cohort_grouped
      )
'''

# Group
sql_fin = '''
      SELECT CohortGroup, MAX(I)/MAX(I) AS '1', MAX(II)/MAX(I) AS '2', MAX(III)/MAX(I) AS '3', MAX(IV)/MAX(I) AS '4', 
                          MAX(V)/MAX(I) AS '5', MAX(VI)/MAX(I) AS '6', MAX(VII)/MAX(I) AS '7', MAX(VIII)/MAX(I) AS '8',
                          MAX(IX)/MAX(I) AS '9', MAX(X)/MAX(I) AS '10', MAX(XV)/MAX(I) AS '11', MAX(XVI)/MAX(I) AS '12', MAX(XVII)/MAX(I) AS '13'
      FROM cohort_pivot
      GROUP BY 1
'''

In [72]:
curs.execute(sql_p_data+','+sql_first_purchase+','+sql_cohort_group+','+sql_cohort_anaysis+','+sql_pivot+sql_fin)

13

In [73]:
cohort_list = []
for data in curs.fetchall():
    cohort_list.append(data)

df_cohort = pd.DataFrame(cohort_list)
df_cohort

,CohortGroup,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2010-12,1.0000,0.3661,0.3232,0.3842,0.3627,0.3977,0.3627,0.3492,0.3537,0.3955,0.3740,0.5028,0.2655
1,2011-01,1.0000,0.2206,0.2662,0.2302,0.3213,0.2878,0.2470,0.2422,0.2998,0.3261,0.3645,0.1175,0.0000
2,2011-02,1.0000,0.1868,0.1868,0.2842,0.2711,0.2474,0.2526,0.2789,0.2474,0.3053,0.0684,0.0000,0.0000
3,2011-03,1.0000,0.1504,0.2522,0.1991,0.2235,0.1681,0.2677,0.2301,0.2788,0.0863,0.0000,0.0000,0.0000
4,2011-04,1.0000,0.2133,0.2033,0.2100,0.1967,0.2267,0.2167,0.2600,0.0733,0.0000,0.0000,0.0000,0.0000
5,2011-05,1.0000,0.1901,0.1725,0.1725,0.2077,0.2324,0.2641,0.0951,0.0000,0.0000,0.0000,0.0000,0.0000
6,2011-06,1.0000,0.1736,0.1570,0.2645,0.2314,0.3347,0.0950,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,2011-07,1.0000,0.1809,0.2074,0.2234,0.2713,0.1117,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,2011-08,1.0000,0.2071,0.2485,0.2426,0.1243,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,2011-09,1.0000,0.2341,0.3010,0.1137,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


---

# RFM Customer Segment with SQL

In [194]:
# 데이터 결측값 제거, 중복값 제거, 음수, 0인 값 제거 (전처리)
sql_p_data = '''
      WITH p_data AS (
            SELECT *, UnitPrice*Quantity AS TotalSum
            FROM online_sales
            WHERE CustomerID IS NOT NULL AND
                  Quantity > 0 AND
                  UnitPrice > 0
            GROUP BY 1,2,3,4,5,6,7,8
      )
'''

# 데이터의 마지막 날짜인 2011-12-09의 바로 다음 날짜인 2011-12-10을 기준으로 Recendency, Frequency, MonetaryValue 계산
sql_rfm = '''
      rfm AS (
            SELECT DATEDIFF(STR_TO_DATE("2011-12-10", '%Y-%m-%d') , DATE(MAX(InvoiceDate))) AS Recency,
                   COUNT(InvoiceNo) AS Frequency,
                   SUM(TotalSum) AS MonetaryValue
            FROM p_data
            GROUP BY CustomerID
      )
'''

# 고객 Segment을 위한 순위 생성
sql_rfm_rank = '''
      rfm_rank AS (
            SELECT *,
                   RANK() OVER (ORDER BY Recency) as Recency_Rank,
                   RANK() OVER (ORDER BY Frequency DESC) as Frequency_Rank,
                   RANK() OVER (ORDER BY MonetaryValue DESC) as MonetaryValue_Rank
            FROM rfm
      )
'''

# 고객 세그먼트(25%, 50%, 75%, 100%) 구간
sql_rfm_seg = '''
      rfm_seg AS (
            SELECT *,
                   CASE WHEN Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS r_tier,
                   CASE WHEN Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS f_tier,
                   CASE WHEN MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS m_tier
            FROM rfm_rank
      )
'''

sql_rfm_score = '''
      SELECT *,
             CONCAT(r_tier, f_tier, m_tier) AS rfm_segment,
             r_tier + f_tier + m_tier AS rfm_score
      FROM rfm_seg
'''

In [195]:
curs.execute(sql_p_data+','+sql_rfm+','+sql_rfm_rank+','+sql_rfm_seg+sql_rfm_score)

4338

In [197]:
rfm_list = []
for data in curs.fetchall():
    rfm_list.append(data)

df_rfm = pd.DataFrame(rfm_list)
df_rfm

""


In [209]:
# 데이터 결측값 제거, 중복값 제거, 음수, 0인 값 제거 (전처리)
sql_p_data = '''
      WITH p_data AS (
            SELECT *, UnitPrice*Quantity AS TotalSum
            FROM online_sales
            WHERE CustomerID IS NOT NULL AND
                  Quantity > 0 AND
                  UnitPrice > 0
            GROUP BY 1,2,3,4,5,6,7,8
      )
'''

# 데이터의 마지막 날짜인 2011-12-09의 바로 다음 날짜인 2011-12-10을 기준으로 Recendency, Frequency, MonetaryValue 계산
sql_rfm = '''
      rfm AS (
            SELECT DATEDIFF(STR_TO_DATE("2011-12-10", '%Y-%m-%d') , DATE(MAX(InvoiceDate))) AS Recency,
                   COUNT(InvoiceNo) AS Frequency,
                   SUM(TotalSum) AS MonetaryValue
            FROM p_data
            GROUP BY CustomerID
      )
'''

# 고객 Segment을 위한 순위 생성
sql_rfm_rank = '''
      rfm_rank AS (
            SELECT *,
                   RANK() OVER (ORDER BY Recency) as Recency_Rank,
                   RANK() OVER (ORDER BY Frequency DESC) as Frequency_Rank,
                   RANK() OVER (ORDER BY MonetaryValue DESC) as MonetaryValue_Rank
            FROM rfm
      )
'''

# 고객 세그먼트(25%, 50%, 75%, 100%) 구간
sql_rfm_seg = '''
      rfm_seg AS (
            SELECT *,
                   CASE WHEN Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND Recency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN Recency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS r_tier,
                   CASE WHEN Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND Frequency_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN Frequency_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS f_tier,
                   CASE WHEN MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.25 THEN 4
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.25 AND MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.5 THEN 3
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.5 AND MonetaryValue_Rank <= (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 2
                        WHEN MonetaryValue_Rank > (SELECT COUNT(*) FROM rfm_rank)*0.75 THEN 1
                        END AS m_tier
            FROM rfm_rank
      )
'''

sql_rfm_score = '''
    rfm_score AS (
        SELECT *,
                CONCAT(r_tier, f_tier, m_tier) AS rfm_segment,
                r_tier + f_tier + m_tier AS rfm_score
        FROM rfm_seg
    )
'''

sql_rfm_segment_by_each = '''
    SELECT rfm_segment,
           AVG(Recency) AS Avg_Recency,
           AVG(Frequency) AS Avg_Recency,
           AVG(MonetaryValue) AS MonetaryValue,
           COUNT(*) AS COUNT
    FROM rfm_score
    GROUP BY rfm_segment
    ORDER BY rfm_segment DESC
'''

sql_rfm_segment_by_score = '''
    SELECT rfm_score,
           AVG(Recency) AS Avg_Recency,
           AVG(Frequency) AS Avg_Recency,
           AVG(MonetaryValue) AS MonetaryValue,
           COUNT(*) AS COUNT
    FROM rfm_score
    GROUP BY rfm_score
    ORDER BY rfm_score DESC
'''

sql_rfm_segment_by_tier = '''
    SELECT CASE WHEN rfm_score > 9 THEN 'GOLD'
                WHEN rfm_score <= 9 AND rfm_score > 5 THEN 'SILVER'
                ELSE 'BRONZE'
                END AS Tier,
           AVG(Recency) AS Avg_Recency,
           AVG(Frequency) AS Avg_Recency,
           AVG(MonetaryValue) AS MonetaryValue,
           COUNT(*) AS COUNT
    FROM rfm_score
    GROUP BY Tier
    ORDER BY MonetaryValue DESC
'''

In [205]:
curs.execute(sql_p_data+','+sql_rfm+','+sql_rfm_rank+','+sql_rfm_seg+','+sql_rfm_score+sql_rfm_segment_by_each)

rfm_list = []
for data in curs.fetchall():
    rfm_list.append(data)

df_rfm = pd.DataFrame(rfm_list)
df_rfm

,rfm_segment,Avg_Recency,.Avg_Recency,MonetaryValue,COUNT
0,444,7.6345,368.7601,9350.605464,446
1,443,8.9326,184.1573,1174.457970,89
2,442,9.5000,133.7857,541.906425,14
3,441,9.0000,113.3333,289.179998,3
4,434,8.8077,75.2051,4420.961510,78
...,...,...,...,...,...
56,121,258.0965,24.9386,200.349473,114
57,114,257.3333,5.5556,19557.439616,9
58,113,237.5652,8.4348,946.234776,23
59,112,258.2000,9.4842,394.684524,95


In [210]:
curs.execute(sql_p_data+','+sql_rfm+','+sql_rfm_rank+','+sql_rfm_seg+','+sql_rfm_score+sql_rfm_segment_by_score)

rfm_list = []
for data in curs.fetchall():
    rfm_list.append(data)

df_rfm = pd.DataFrame(rfm_list)
df_rfm

,rfm_score,Avg_Recency,.Avg_Recency,MonetaryValue,COUNT
0,12,7.6345,368.7601,9350.605464,446
1,11,21.6324,187.7763,4045.994888,389
2,10,32.8410,110.0691,2048.967086,434
3,9,46.9343,78.8929,1798.577501,411
4,8,64.4617,55.6638,979.717909,470
5,7,81.6793,37.7350,754.843182,449
6,6,97.4547,27.7608,813.398031,464
7,5,155.6360,20.8084,365.117739,522
8,4,179.4597,13.5948,233.125427,385
9,3,260.3397,7.8886,155.777934,368


In [211]:
curs.execute(sql_p_data+','+sql_rfm+','+sql_rfm_rank+','+sql_rfm_seg+','+sql_rfm_score+sql_rfm_segment_by_tier)

rfm_list = []
for data in curs.fetchall():
    rfm_list.append(data)

df_rfm = pd.DataFrame(rfm_list)
df_rfm

,Tier,Avg_Recency,.Avg_Recency,MonetaryValue,COUNT
0,GOLD,20.5461,224.8085,5227.355212,1269
1,SILVER,73.2887,49.2815,1068.017862,1794
2,BRONZE,193.0502,14.9012,264.840023,1275
